# Earth Engine

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davemlz/spyndex/blob/main/docs/tutorials/ee.ipynb)
![level7](https://raw.githubusercontent.com/davemlz/spyndex/main/docs/_static/level7.png)

New level: Level 7 - `spyndex + Google Earth Engine`!

Remember to install `spyndex`!

In [ ]:
!pip install -U spyndex

Now, let's start!

First, import `spyndex` and `Earth Engine`:

> We also import `eemont` and `geemap` to make things easier.

In [21]:
import spyndex
import ee, eemont, geemap

And remember to initialize `Earth Engine`:

In [22]:
Map = geemap.Map()

## `ee.Number`

Just as Python Built-In types, the `ee.Number` can be used with `spyndex`!

In [23]:
NIR = ee.Number(0.678)
RED = ee.Number(0.123)

Let's check our data types:

In [27]:
print(f"NIR type: {type(NIR)}, value: {NIR.getInfo()}")
print(f"RED type: {type(RED)}, value: {RED.getInfo()}")

NIR type: <class 'ee.ee_number.Number'>, value: 0.678
RED type: <class 'ee.ee_number.Number'>, value: 0.123


And let's compute the `NDVI`.

In [28]:
idx = spyndex.computeIndex("NDVI",{"N": NIR, "R": RED})

And our result is also an `ee.Number`!

In [29]:
print(f"idx type: {type(idx)}, value: {idx.getInfo()}")

idx type: <class 'ee.ee_number.Number'>, value: 0.6928838951310862


## `ee.Image`

All spectral indices from `Awesome Spectral Indices` can be computed automatically in Python for `Earth Engine` by using `eemont` for most datasets (Sentinel-2, MODIS, Landsat, etc.). However, some datasets are not included. NAIP is a good example, let's use a NAIP image!

In [3]:
img = ee.Image("USDA/NAIP/DOQQ/m_3009057_ne_15_1_20170910")

NAIP images values go from 0 to 255, so, let's correct that!

In [4]:
img = img / 255.0

Let's check the image metadata:

In [5]:
img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'R',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 1},
   'dimensions': [6753, 7640],
   'crs': 'EPSG:26915',
   'crs_transform': [1, 0, 698395, 0, -1, 3334843]},
  {'id': 'G',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 1},
   'dimensions': [6753, 7640],
   'crs': 'EPSG:26915',
   'crs_transform': [1, 0, 698395, 0, -1, 3334843]},
  {'id': 'B',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 1},
   'dimensions': [6753, 7640],
   'crs': 'EPSG:26915',
   'crs_transform': [1, 0, 698395, 0, -1, 3334843]},
  {'id': 'N',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 1},
   'dimensions': [6753, 7640],
   'crs': 'EPSG:26915',
   'crs_transform': [1, 0, 698395, 0, -1, 3334843]}]}

Great! We have 4 bands (R, G, B, N) and all of them go now from 0 to 1.

Let's compute some indices!

In [6]:
idx = spyndex.computeIndex(
    index = ["VARI","ExG","NDVI"],
    params = {
        "R": img["R"],
        "G": img["G"],
        "B": img["B"],
        "N": img["N"]
    }
)

If we check the indices, we'll see that the result is another `ee.Image`!

In [7]:
print(f"idx type: {type(idx)}")

idx type: <class 'ee.image.Image'>


Now, let's add those indices as new bands in the original image.

In [8]:
img = img.addBands(idx)

If we check our bands, we'll see that the new bands (indices) were added:

In [9]:
img.bandNames().getInfo()

['R', 'G', 'B', 'N', 'VARI', 'ExG', 'NDVI']

Let's visualize evrything interactively with `geemap`!

First, import the colormaps:

In [15]:
import geemap.colormaps as cm

And then, plot everything!

In [17]:
Map = geemap.Map()
Map.addLayer(img[["R","G","B"]],{"min":0,"max":0.3},"RGB")
Map.addLayer(img["VARI"],{"palette":cm.palettes.ndvi},"VARI")
Map.addLayer(img["ExG"],{"palette":cm.palettes.ndvi},"ExG")
Map.addLayer(img["NDVI"],{"palette":cm.palettes.ndvi},"NDVI")
Map.centerObject(img,13)
Map

Map(center=[30.09375197359924, -90.90623221580759], controls=(WidgetControl(options=['position', 'transparent_…

Amazing!!!! :)